In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv020649/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

# Normal join

In [2]:
spark

In [3]:
orderRdd = spark.sparkContext.textFile("/public/trendytech/orders/orders_1gb.csv")
customerRdd = spark.sparkContext.textFile("/public/trendytech/retail_db/customers/part-00000")
# order 1gb 8 block 8p
#  customers 1 mb -1 block 1p
# customer_id PK
# data need to shufferled to join data with same key

In [4]:
orderMapped = orderRdd.map(lambda x: (x.split(",")[2],x.split(",")[3]))

In [5]:
customerMapped = customerRdd.map(lambda x: (x.split(",")[0],x.split(",")[0]))

In [6]:
joinedRdd = customerMapped.join(orderMapped)

In [7]:
# joinedRdd.saveAsTextFile("data/orders_joined")

In [8]:
# spark.stop()

# Better way to join

challange: data shuffeled a lot
order table big
customer small
what can be done: customer table can be copied to all npde/partition


When you call collect() on a pair RDD, Spark automatically treats each tuple as a key-value pair

In [9]:
customerBroadcast = spark.sparkContext.broadcast(customerMapped.collect())

In [10]:
def get_pincode(customer_id):
    try:
        return customerBroadcast.value[customer_id]
    except:
        return "-1"
# return customerBroadcast.value[customer_id][0] for tripple value

In [11]:
joinRdd = orderMapped.map(lambda x: (get_pincode(int(x[0])),x[1]))

In [2]:
joinRdd.saveAsTextFile("data/orders_joined1")

NameError: name 'joinRdd' is not defined